In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# Declare a Base using `automap_base()`
Base = automap_base()

# reflect an existing database into a new model
# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Display the row's columns and data in dictionary format
first_row_station = session.query(station).first()
first_row_station.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f8e90045460>,
 'longitude': -157.8168,
 'id': 1,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397',
 'latitude': 21.2716,
 'elevation': 3.0}

In [10]:
# Display the row's columns and data in dictionary format
first_row_measurement = session.query(measurement).first()
first_row_measurement.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x7f8e90045670>,
 'date': '2010-01-01',
 'id': 1,
 'tobs': 65.0,
 'prcp': 0.08,
 'station': 'USC00519397'}

In [11]:
# Find the most recent date in the data set.
recent_date = session.query(measurement.date).order_by(measurement.date.desc()).first()
recent_date

('2017-08-23',)

In [12]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [ ]:
# Using the inspector to print the column names within the 'dow' table and its types
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
last_12_months = session.query(measurement.date, measurement.prcp).order_by(measurement.date.asc()).all()
last_12_months

In [ ]:
# Starting from the most recent data point in the database. 

#date_object = datetime.strptime(date_string, "%d %B, %Y")

recent_data_point = dt.datetime.strptime(recent_date[0], "%Y-%m-%d")
recent_data_point

# printing calculated past_dates
print('recent_data_point:', str(recent_data_point))


In [ ]:
# Calculate the date one year from the last date in data set.

past_date_one_yr = recent_data_point - timedelta(days=365)
past_date_one_yr

# printing calculated past_dates
print('past_date_one_yr:', str(past_date_one_yr))


In [ ]:
# Perform a query to retrieve the data and precipitation scores

prcp_scores_query = session.query(measurement.date, measurement.prcp).filter(measurement.date > past_date_one_yr).order_by(measurement.date.desc()).all()
#prcp_scores_query

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
prcp_query_results = pd.DataFrame(prcp_scores_query, columns=['Date', 'Percipitation'])

prcp_query_results.set_index('Date', inplace=True)

prcp_query_results

In [ ]:
# Sort the dataframe by date

prcp_query_results = pd.DataFrame.sort_index(prcp_query_results)
prcp_query_results

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

prcp_query_results.plot.bar(figsize=(15,15), width=20)
plt.ylabel('Inches')
plt.locator_params(axis='x', nbins=9)
plt.ylim(0,max(prcp_query_results['Percipitation']))
plt.show();


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

prcp_query_results.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(func.count(station.id)).all()
station_count

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

active_station_count = session.query(measurement.station, func.count(measurement.id)).group_by(measurement.station).order_by(func.count(measurement.id).desc()).all()

active_station_count


In [ ]:
#Answer the following question: which station id has the greatest number of observations?
print(f"The station ID with the greatest number of observations is USC00519281.")

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
sel = [func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)]
                
session.query(*sel).filter(measurement.station=='USC00519281').all()



In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station

temp_last_12_months = session.query(measurement.date, measurement.tobs).filter(measurement.station=='USC00519281').all()
# temp_last_12_months

temp_list = []
for temp in temp_last_12_months:
    temp_list.append(temp[1])


In [ ]:
 # and plot the results as a histogram

plt.hist(temp_list, color='blue', bins=12)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


# Close session

In [ ]:
# Close Session
session.close()